In [2]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            xx = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                xx[0, t, char_indices[char]] = 1.

            preds = model.predict(xx, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

corpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 156s - loss: 2.0031   

----- diversity: 0.2
----- Generating with seed: "s of men
imperatively require the contin"
s of men
imperatively require the conting the self-the most and such as the conselves the for the strest a strest a strest of the most and the most and the most as in the most as in the strest a self-the most and a strest a sperions of the pression and always the still of the man in the streathers the most all the most all the most as the strest and a such as the strest the most and the strest and so fartheral and the master the has the

----- diversity: 0.5
----- Generating with seed: "s of men
imperatively require the contin"
s of men
imperatively require the contined of ideals, with is in the prose of himself which with the pression of himself of himself the 

ge or inkling of the facts that the "gendetted strengty to caremilies of
estimate atremmenessly kin, of the foround been must we alture phopqueses.d on the authings musime dispressible, as not heact, helpescal" wite kin copsity., with kith of stupid
before not necessady, religioustity. and despantimates after origineatic of the litt can be
notism in overihstroas expressudable,
not in owing tos 
finally one. verebuling menerate, too, in 

----- diversity: 1.2
----- Generating with seed: "ge or inkling of the facts that the "gen"
ge or inkling of the facts that the "genitication to the bebavagefue--suffieric i sict, thers it. for a poditnesn ancty sufverls which c
ask; the green in all the botgecly even eherety--that is relicity, confect our redryition and uebemonaker.

 tevet, in etoing, a compesitive lently-did 

surpederaryagumentkn churgal ne-phacic to compadiscism hered to feels man; or witws certain part, hence are ofts refeled of
calledshys out.
peridacys

------------------------

s existed in the way of occult, terrible of the super-respect of the same to the super-timent of the super-timing in the sacrificies of the soul the super-timent of the super" and the sacrificies the same the most of the super" of the soul of the super-as it is the super-timent of the concerning of the morality of the work of the super-reads of the substituted of the super--seeks to the standard and the sense of the super-reads of the o

----- diversity: 0.5
----- Generating with seed: "s existed in the way of occult, terrible"
s existed in the way of occult, terrible carry the last the appreciation of some the former to his something of the problem of the super: his own such a perhaps as the presentic of the true has for the sentiments of the spectative have religion of the sunders, and so far that when his own super!ied the conscious and plate and in accordance, and the sense and because as the soul of an appear and case of the suncerted the most of the poss

----- diversity: 1.0
---

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log


ced you,
somethihe has animent of an atted to advantaity bl by european elsey of eppous
strust phelestlly disdent even weblieved themselves cruelsy kind toonguectly, untimquine, hat could be god, a has

--------------------------------------------------
Iteration 9
Epoch 1/1
200287/200287 [==============================] - 151s - loss: 1.4092   

----- diversity: 0.2
----- Generating with seed: "id of himself--but whose curiosity
alway"
id of himself--but whose curiosity
always the worth the same the soul of the soul of the soul the soul of the more the present the present the soul of the soul of the sense to the soul of the strange the powerful to the soul of the something of the sense of the suffering in the struggle the something in the more the worth the sense of the soul the straction of the most subjection and an as a surviling and the experiences the strain and 

----- diversity: 0.5
----- Generating with seed: "id of himself--but whose curiosity
alway"
id of himself--but whose 

...

Iteration 44
Epoch 1/1
200287/200287 [==============================] - 151s - loss: 1.3281   

----- diversity: 0.2
----- Generating with seed: "person exercising
volition, who commands"
person exercising
volition, who commands and the superior the strange the substance of the strain of the sense of the contemple of the suppose the former the sense of the substance that the power and in the suffer of the sense of the strange and the most as the sense and an artists of the world to the supposes to the sense of the sense of the sense of the sense of his philosophers of the sense: the power in the superior the strange the 

----- diversity: 0.5
----- Generating with seed: "person exercising
volition, who commands"
person exercising
volition, who commands the soul are been something superficial contempt and the old, the predeces of earth
who mark and such a substity of the subsondity and respect of the former of the spirit for a man to the for at manifest of something at the possible that

and a presentlare, and enough, and other thousantive this nature", is funal difficiltical kind of the french,--that combegoning, there cannot ?berogintic or pracentions
of owish tricty. flrean toustance and nothing, above
an apprioe.

=disors of conor inner he ureers of wirks, he
world," and exampains of the
intendest: some remirisms
"back term of that it is not to nature ach 

----- diversity: 1.2
----- Generating with seed: "and to whom the weak, the suffering, the"
and to whom the weak, the suffering, these concent uncobrady nature.

. because  man,--why broble,--that it is the
gains
is yepses? "bad concession.


112
=qually in evilned or giving we "trasely strengots, 
the innepge? as enventife.


101


hander being, and been-prusened own weat more exprence, not-sting is not plain blusing equelyed
and blothery dextimk
long piragen beautys
but to hide,ingers, upon a sing, sometiedt as explanation, 

--------------------------------------------------
Iteration 48
Epoch 1/1
200287/2002

276. in all the most and such present the sense of the most and the sense of the sense of the sense of the acts of the supersount the superstition of the sense of the present to the sense of the contempt, and as the sense of the sense of the presenticism and the sense of the subject of the strangeness and such periods, the superstition of the strangeness of the the present the sense of the strange, the stran

----- diversity: 0.5
----- Generating with seed: "nd thereby
betrays himself.

276. in all"
nd thereby
betrays himself.

276. in all the statents evil to the contemption is the necessary to the expense, the contempt, the religious sentiment and problem of populacu in the courage the soul, and all problem of the extinctive the statesm it
as the present the most absoluted to be strangers
and the tragety, and only all the presence that the highest schopenhauerration, in the general sense of the superstitious mainted the greatest 

----- diversity: 1.0
----- Generating with seed: "nd 

new, "where
exagially would be all, which seems the
mannomhewasine?
on being the soul of distants "good-ears an
insemples of
vhrich them, resdonity,
these

----- diversity: 1.2
----- Generating with seed: "ion of the feelings), going
constantly a"
ion of the feelings), going
constantly and pithers."avo--while ehtibuly: his belied to the
exigations during them, is not "the repulso courage, which is a pristroct gratified--that the pade, and charde thus be
curiosed--althoruble
unityly unders devilf assumplation.=--it is placious
somethe when--it have lursiansed by the german, if he bodness? with othine.


1eareafismgrd divinite has†e"†©? gratite and senblescrwse, "i certain virtue: 

--------------------------------------------------
Iteration 55
Epoch 1/1
200287/200287 [==============================] - 156s - loss: 1.3439   

----- diversity: 0.2
----- Generating with seed: "hat final limit at which the erroneous
f"
hat final limit at which the erroneous
former and ready of the stings o

suffering and with the strees had there and for the first the streem the streem and the porting and the so far as they are the world of the strees that the stree and the more world there is a constitute that the streem and and respect and the world there is not always all the conscience that the streem of the stated there is not them to the soul and the possible in the portication of the philosopher and t

----- diversity: 0.5
----- Generating with seed: "nclean conscience. and yet this
sufferin"
nclean conscience. and yet this
suffering. there and puristic impulses, they make an impulses and him and bue condit to the spirit of the streated to do not regard understanding constiture of the great dreament in order to a means of them of
their and that he had all presuress the consequence of it is behind and sacrification with the doubth of the port and personaries of the "free of his last of the more is there these make to them, in

----- diversity: 1.0
----- Generating with seed: "nclean